In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow
from matplotlib.image import imread
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Reshape
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, Conv2DTranspose
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import cv2
%matplotlib inline

In [ ]:
path = "../input/covid19-radiography-dataset/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset"
os.listdir(path)

In [ ]:
covid = path+"/COVID/"
normal = path+"/Normal/"
opacity = path+"/Lung_Opacity/"
pneumonia = path+"/Viral Pneumonia/"

In [ ]:
os.listdir(covid)[0]

In [ ]:
plt.imshow(imread(covid+"COVID-1.png"))

In [ ]:
imread(covid+"COVID-1.png")

In [ ]:
os.listdir(normal)[0]

In [ ]:
plt.imshow(imread(normal+"Normal-6196.png"))

In [ ]:
os.listdir(opacity)[0]

In [ ]:
plt.imshow(imread(opacity+"Lung_Opacity-129.png"))

In [ ]:
os.listdir(pneumonia)[0]

In [ ]:
plt.imshow(imread(pneumonia+"Viral Pneumonia-1186.png"))

In [ ]:
imread(pneumonia+"Viral Pneumonia-1186.png").shape

In [ ]:
len(os.listdir(covid))

In [ ]:
len(os.listdir(normal))

In [ ]:
len(os.listdir(opacity))

In [ ]:
len(os.listdir(pneumonia))

In [ ]:
print(imread(covid+os.listdir(covid)[0]).shape)
print(imread(normal+os.listdir(normal)[0]).shape)
print(imread(opacity+os.listdir(opacity)[0]).shape)
print(imread(pneumonia+os.listdir(pneumonia)[0]).shape)

In [ ]:
image_shape=(299, 299, 1)

In [ ]:
def to_grayscale(img):
    img = tensorflow.image.rgb_to_grayscale(img)
    return img

In [ ]:
image_gen = ImageDataGenerator(width_shift_range=0.1, 
                              height_shift_range=0.1, 
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest",
                              horizontal_flip=True,
                              validation_split=0.25)

In [ ]:
image_gen.flow_from_directory(path, classes=["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"])

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=image_shape, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=image_shape, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), input_shape=image_shape, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), input_shape=image_shape, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), input_shape=image_shape, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(4, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=5,
                           restore_best_weights=True, verbose=1)

In [ ]:
batch_size=32
train_gen = image_gen.flow_from_directory(path, target_size=image_shape[:2], color_mode="grayscale", 
                                         batch_size=batch_size, class_mode="categorical", 
                                          classes=["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"], 
                                          shuffle=True)

test_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                                 directory=path,
                                                 shuffle=False,
                                                 color_mode="grayscale",
                                                 target_size=image_shape[:2], 
                                                 subset="validation",
                                                 class_mode='categorical', 
                                         classes=["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"])

In [ ]:
train_gen.class_indices

In [ ]:
model.fit(train_gen, validation_data=test_gen, epochs=10, callbacks=[early_stop])

In [ ]:
model.save("disease_classification.h5")

In [ ]:
metrics = pd.DataFrame(model.history.history)
metrics

In [ ]:
metrics[["loss", "val_loss"]].plot()

In [ ]:
metrics[["accuracy", "val_accuracy"]].plot()

In [ ]:
pred_probabilities = model.predict_generator(test_gen)

In [ ]:
pred_probabilities

In [ ]:
predictions = np.argmax(pred_probabilities,axis=1)
predictions

In [ ]:
np.concatenate((predictions.reshape(len(predictions), 1), 
                test_gen.classes.reshape(len(test_gen.classes), 1)), axis=1)

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(confusion_matrix(test_gen.classes, predictions), annot=True, cmap="viridis")

In [ ]:
print(classification_report(test_gen.classes, predictions))

In [ ]:
# inception.fit(train_gen, validation_data=test_gen, epochs=10, callbacks=[early_stop])

In [ ]:
vgg = VGG16(include_top=True, weights=None, input_shape=image_shape, pooling="max", 
           classes=4, classifier_activation="softmax")
vgg.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
vgg.summary()

In [ ]:
vgg.fit(train_gen, validation_data=test_gen, epochs=10, callbacks=[early_stop])

In [ ]:
xception = Xception(include_top=True, weights=None, input_shape=image_shape, pooling="max",
                   classes=4, classifier_activation="softmax")

xception.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
xception.summary()

In [ ]:
xception.fit(train_gen, validation_data=test_gen, epochs=10, callbacks=[early_stop])

In [ ]:
probabilities = xception.predict_generator(test_gen)
predictions = np.argmax(probabilities, axis=1)
print(classification_report(test_gen.classes, predictions))

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(confusion_matrix(test_gen.classes, predictions), annot=True, cmap="viridis")

In [ ]:
codings_size=3200

In [ ]:
generator = Sequential()
generator.add(Dense(39 * 39 * 128, input_shape=[codings_size]))
generator.add(Reshape([39, 39, 128]))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding="same",
                                 activation="relu"))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Conv2DTranspose(1, kernel_size=5, strides=2, padding="same",
                                 activation="tanh"))

In [ ]:
generator.summary()

In [ ]:
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, padding="same",
                        activation=LeakyReLU(0.3),
                        input_shape=[156, 156, 1]))
discriminator.add(Dropout(0.5))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same",
                        activation=LeakyReLU(0.3)))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(Dropout(0.5))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation="sigmoid"))

In [ ]:
GAN = Sequential([generator, discriminator])

In [ ]:
discriminator.compile(optimizer="adam", loss="binary_crossentropy")
discriminator.trainable=False

In [ ]:
GAN.compile(optimizer="adam", loss="binary_crossentropy")

In [ ]:
batch_size=32
data = tf.data.Dataset.from_tensor_slices(covid_images).shuffle(buffer_size=1000)

In [ ]:
data = data.batch(batch_size, drop_remainder=True).prefetch(1)

In [ ]:
epochs=400
generator, discriminator = GAN.layers
for epoch in range(epochs):
    print(f"Currently on Epoch {epoch+1}")
    i = 0
    for X_batch in data:
        ## TRAINING THE DISCRIMINATOR ######
        noise = tf.random.normal(shape=[batch_size, codings_size])
        gen_images = generator(noise)
        gen_images = tf.reshape(gen_images, [32, 156, 156, 1])
        X_fake_vs_real = tf.concat([gen_images, tf.dtypes.cast(X_batch,tf.float32)], axis=0)
        y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
        
        discriminator.trainable = True
        
        discriminator.train_on_batch(X_fake_vs_real, y1)
        
        ## TRAINING THE GENERATOR ##
        noise = tf.random.normal(shape=[batch_size, codings_size])
        
        y2 = tf.constant([[1.]] * batch_size)
        
        discriminator.trainable = False
        
        GAN.train_on_batch(noise, y2)

In [ ]:
noise = tf.random.normal(shape=[5, codings_size])

In [ ]:
plt.imshow(noise)

In [ ]:
images = generator(noise)
for i in images:
    plt.imshow(i.numpy().reshape(156, 156))
    plt.show()

In [ ]:
def prediction(model, img_path):
    classes = dict(enumerate(train_gen.class_indices))
    return classes[model.predict_classes(imread(img_path).reshape(-1, 299, 299, 1))[0]]

**DEPLOYMENT**

In [ ]:
classifier = load_model("disease_classification.h5")

def prediction(model, img_path):
    classes = dict(enumerate(train_gen.class_indices))
    return classes[model.predict_classes(imread(img_path).reshape(-1, 299, 299, 1))[0]]